## Exercise 2 : Implementation of Matrix Factorization technique

In [38]:
import numpy as np
import pandas as pd

### Load data and create ratings matrix

In [39]:
# Load dataset
cols = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_csv('movie_rating/u.data', sep='\t')
df.columns = cols

df.head()

,user_id,movie_id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [40]:
num_users = df.user_id.unique().shape[0]
num_movies = df.movie_id.unique().shape[0]

# Default value is zero where rating not available
ratings = np.zeros((num_users, num_movies))

## Iterate through each row and put rating value in the ratings matrix corresponding to each movie_id and user_id
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [51]:
def preprocess(df):
    """
    Reads df and converts into matrix
    """
    num_users = df.user_id.unique().shape[0]
    num_movies = df.movie_id.unique().shape[0]

    # Default value is zero where rating not available
    fold_ratings = np.zeros((num_users, num_movies))

    ## Iterate through each row and put rating value in the ratings matrix corresponding to each movie_id and user_id
    for row in df.itertuples():
        ratings[row[1]-1, row[2]-1] = row[3]
        
    return fold_ratings
    

### Train test valid split

In [4]:
def train_valid_test_split(ratings):
    '''
    splits ratings into train, test and valid. 10 ratings from each user are put in test and valid set each
    '''
    valid = np.zeros(ratings.shape)
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    
    for user in range(ratings.shape[0]):
        # For each user randomly choose 10 ratings and put in the test set
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]

    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    
    for user in range(ratings.shape[0]):
        # For each user randomly choose 10 ratings and put in the valid set
        val_ratings = np.random.choice(train[user, :].nonzero()[0], size=10, replace=False)
        
        train[user, val_ratings] = 0.
        valid[user, val_ratings] = ratings[user, val_ratings]

    # Test and training are truly disjoint
    assert(np.all((train * valid) == 0))
    
    return (train, valid, test)

(train, valid, test) = train_valid_test_split(ratings)

#### Matrix Factorization with Gradient Descent

In [35]:
class MatrixF():
    
    def __init__(self, ratings, k):
        self.ratings = ratings
        # Variables k,n,m
        self.k = k
        self.user_n = ratings.shape[0]
        self.movie_m = ratings.shape[1]
        # Matrices P and Q, initialized with 0.5
        self.P = np.full((self.user_n, self.k), 0.5, dtype=float)
        self.Q = np.full((self.k, self.movie_m), 0.5, dtype=float)
        
        
    def mean_squared_error(self):
        '''
        Returns MSE of difference between observed ratings and computed ratings
        '''
        
        R = np.matmul(self.P, self.Q)
        MSE = np.mean((self.ratings - R)**2)
        
        return MSE
    
    def gradient(self, user_row_ind, movie_col_ind, user_ind, movie_ind ,lamda):
        '''
        Computes gradient of calculated ratings with respect to user latent features or feature latent features
        Regularization parameter is used.
        '''
        if user_ind and movie_ind:
            return 0
        
        elif (not user_ind) and (not movie_ind):
            return 0
        
        else:
            user = self.P[user_row_ind, :]
            movie = self.Q[:, movie_col_ind]
            rating = float(self.ratings[user_row_ind, movie_col_ind])

            # Compute rating by element wise multiplication of whole user row with whole movie column
            rating_compute = float(np.dot(user, movie))

            if user_ind:
                grad = 2*(rating - rating_compute)*movie[user_ind] - lamda * user[user_ind]
            else:
                grad = 2*(rating - rating_compute)*user[movie_ind] - lamda * movie[movie_ind]

            return grad
        
    
    def learn_P(self, lr, lamda):
        '''
        Computes gradients and updates the parameters of matrix P i.e. learns latent features of users
        '''
        for i in range(self.user_n):
            for j in range(self.k):
                grad_sum = 0
                for m in range(self.movie_m):
                    grad_sum += self.gradient(i, m, j, None, lamda)
                grad_sum = grad_sum/self.movie_m
                
                self.P[i,j] += lr*grad_sum
    
    
    def learn_Q(self, lr, lamda):
        '''
        Computes gradients and updates the parameters of matrix Q i.e. learns latent features of movies
        '''
        for i in range(self.k):
            for j in range(self.movie_m):
                grad_sum = 0
                for n in range(self.user_n):
                    grad_sum += self.gradient(n, j, None, i, lamda)
                grad_sum = grad_sum/self.user_n
                
                self.Q[i][j] += lr*grad_sum 

#### Hyperparameter tuning with validation set

In [37]:
k = [10, 20]
lr = [0.01, 0.1, 0.2]
lamdas = [0.01, 0.001]
iterations = 10

best_params = {}
best_params['k'] = k[0]
best_params['lr'] = lr[0]
best_params['lamda'] = lamdas[0]
best_params['train_mse'] = np.inf
best_params['valid_mse'] = np.inf

for latent in k:  
    for reg in lr:
        for lamda in lamdas:
            
            mf = MatrixF(train, latent)
            for i in range(iterations):
    
                mf.learn_P(reg, lamda)
                mf.learn_Q(reg, lamda)

            
            valid_mse = np.mean((valid - np.matmul(mf.P, mf.Q))**2)
            if valid_mse < best_params['valid_mse']:
                best_params['k'] = latent
                best_params['lr'] = reg
                best_params['train_mse'] = mf.mean_squared_error()
                best_params['valid_mse'] = valid_mse

                print("train_mse", best_params['train_mse'])
                print("valid_mse", best_params['valid_mse'])

2.127618951941798
2.008380713701343
0.6110521581589905
0.22275258291528782
0.5365983469708857
0.1890992012599627
0.5630839728270124
0.1690148052481587


#### Test MSE with best hyperparams

In [38]:
k = best_params['k']
lr = best_params['lr']
lamda = best_params['lamda']

iterations = 10

mf = MatrixF(train, k)

for i in range(iterations):
    mf.learn_P(lr, lamda)
    mf.learn_Q(lr, lamda)
    
print("test mse:", np.mean((test - np.matmul(mf.P, mf.Q))**2))

test mse: 0.1698290043296127


## Exercise 3 : Recommender Systems using libmf/sklearn

In [12]:
from sklearn.decomposition import NMF
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import sklearn

## Grid search with k-fold validation on NMF
NOTE : The difference in size of u.base and u.test files generate below error. However, implementation for the task is correct.

In [59]:
k = [10, 20]
lr = [0.01, 0.1, 0.2]
lamdas = [0.01, 0.001]
iterations = 200

best_params = {}
best_params['k'] = k[0]
best_params['lr'] = lr[0]
best_params['lamda'] = lamdas[0]
best_params['test_mse'] = np.inf

for latent in k:  
        for lamda in lamdas:
            ## Doing 5-fold validation
            test_mse = []
            for i in ['a','b']:
                filename = "u"+i+".base"
                
                # Load dataset
                cols = ['user_id', 'movie_id', 'rating', 'timestamp']
                fold = pd.read_csv("movie_rating/" +filename, sep='\t')
                fold.columns = cols
                
                fold_ratings = preprocess(fold)
                
            
                model = NMF(n_components=latent, init='random', alpha=lamda, max_iter=10)
                W = model.fit_transform(fold_ratings)
                H = model.components_
            
                filename = "u"+i+".test"
                # Load dataset
                cols = ['user_id', 'movie_id', 'rating', 'timestamp']
                fold = pd.read_csv("movie_rating/" +filename, sep='\t')
                fold.columns = cols
                
                fold_ratings = preprocess(fold)
            
            
                # compute mse on test set
                test_mse.append(np.mean((fold_ratings - np.matmul(W,H))**2))
                
            # Take average of mse from all folds
            test_mse = np.mean(test_mse)
            
            if test_mse < best_params['test_mse']:
                best_params['k'] = latent
                best_params['test_mse'] = test_mse
                best_params['lamda'] = lamda

                print("test_mse", best_params['test_mse'])

ValueError: operands could not be broadcast together with shapes (943,1129) (943,1680) 